In [ ]:
from transformers import BertModel, BertTokenizer
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from .data.squad_dataloader import SquadDataset
from .data.data_utils import squad_collate_fn
from .run_fine_tuning.squad_args import squad_args
from .run_fine_tuning.run_train_squad import train_epoch
from .run_fine_tuning.run_valid_squad import eval_epoch

import numpy as np
import random
from tqdm.notebook import tqdm
import pandas as pd
from matplotlib import pyplot as plt

args = squad_args()
d_hidden = args.d_hidden
device = args.device
batch_size = args.batch_size
n_epoch = args.n_epoch
learning_rate = args.learning_rate


In [ ]:

#Load train and test dataset
train_dataset = SquadDataset('./SQuAD_train.json')
train_loader = DataLoader(
    train_dataset, 
    batch_size= batch_size, 
    shuffle=True, 
    collate_fn=squad_collate_fn, 
    drop_last= True
    )

valid_dataset = SquadDataset('./SQuAD_test.json')
valid_loader = DataLoader(
    valid_dataset,
    batch_size= batch_size, 
    shuffle=True, 
    collate_fn=squad_collate_fn, 
    drop_last= True
    )


In [ ]:
#Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#Load model
class QuestionAnswering(nn.Module):
    def __init__(self, d_hidden):
        super().__init__()

        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.projection = nn.Linear(d_hidden, 2)

    def get_token_emb(self):
        return self.model.get_input_embeddings()

    def forward(self, inputs):
        bert_outputs = self.model(inputs)[0]
        pred_scores = self.projection(bert_outputs)
        return pred_scores

model = QuestionAnswering(device, d_hidden)
model.to(device)

In [ ]:

#Criterion, Optimizer and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(
    model.parameters(), 
    lr = learning_rate, 
    correct_bias = False
    ) #default lr  = 5e-5, eps = 1e-8
total_steps = len(train_loader) * n_epoch
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps = 0, 
    num_training_steps = total_steps
    )

#random seed
seed_val = 100
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#gradient init
model.zero_grad()


In [ ]:
losses = []
scores = []

for epoch in range(n_epoch):
    #train_epoch
    model.train()
    loss_epoch = []

    with tqdm(total = len(train_loader), desc = f"Train {epoch}") as pbar:
        for i, value in enumerate(train_loader):
            _, answer_indices, bert_inputs = map(lambda x:x.to(device), value)

            loss = train_epoch(
                model, 
                criterion, 
                optimizer, 
                answer_indices,
                bert_inputs
                )
            loss_epoch.append(loss)

            scheduler.step()

            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss:.3f} ({np.mean(loss_epoch):.3f})")

    losses.append(np.mean(loss_epoch))

    #valid_epoch
    model.eval()
    score_epoch = []

    with tqdm(total = len(valid_loader), desc = f"Valid {epoch}") as pbar:
        for i, value in enumerate(valid_loader): 
            answers, _, bert_inputs = map(lambda x:x.to(device), value)

            score = eval_epoch(
                model, 
                answers, 
                bert_inputs, 
                batch_size, 
                tokenizer
                )
            score_epoch.append(score)

            pbar.update(1)
            pbar.set_postfix_str(f"Acc: {score:.3f} ({np.mean(score_epoch):.3f})")

    scores.append(np.mean(score_epoch))


In [1]:
#loss and score graph
losses = [1.0, 2.0, 3.0]
scores = [3.1, 2.1, 5.7]
data = {
    "loss" : losses,
    "score" : scores
}
df = pd.DataFrame(data)
display(df)

plt.figure(figsize = [12, 4])
plt.plot(losses, label="loss")
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.show()

,loss,score
0,1.0,3.1
1,2.0,2.1
2,3.0,5.7


NameError: name 'plt' is not defined